# notes

## projections

- 4326 # wgs84 (for H3 compatibility)
    - H3 **does not use** wgs84/epsg4326, but it **does use** the "authalic radius" from that standard
    - lat/lng
- 3857 # web mercator (for plotting), but in meters!
- 2263 # new york (just what's used in the example below)

## geoviews

- tile options: https://github.com/holoviz/geoviews/blob/main/geoviews/tile_sources.py
    - also this, but i don't see a good way to programatically get all the options: https://geoviews.org/gallery/matplotlib/tile_sources.html 
- active tools: https://stackoverflow.com/questions/57767169/set-box-zoom-or-pan-or-wheel-zoom-as-default-in-using-holoviews-or-hvplot
- bokeh pallets: https://docs.bokeh.org/en/latest/docs/reference/palettes.html
- Bokeh "tap" tool seems to do something on this website, but i can't get the same functionality (isolating a single geometry) locally: https://geoviews.org/gallery/bokeh/new_york_boroughs.html

## folium

- seems to be most popular, but they make you manually determine the center and zoom level!
    - too much work. geoviews does it automatically
    - https://geopandas.org/en/stable/gallery/polygon_plotting_with_folium.html

In [ ]:
import h3
import geopandas
import geodatasets
import contextily as cx
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
    >>> h3.H3Poly(
    ...     [(37.68, -122.54), (37.68, -122.34), (37.82, -122.34), (37.82, -122.54)],
    ...     [(37.76, -122.51), (37.76, -122.44), (37.81, -122.51)],
    ...     [(37.71, -122.43), (37.71, -122.37), (37.73, -122.37), (37.75, -122.41),
    ...      (37.73, -122.43)],
    ... )

In [ ]:
df = geopandas.read_file(geodatasets.get_path('nybb'))
df.crs

In [ ]:
df

In [ ]:
def foo(df, epsg):
    df = df.to_crs(epsg=epsg)
    
    ax = df.plot(figsize=(10, 10), alpha=0.5, edgecolor='k', column='BoroName', categorical=True, legend=True, legend_kwds={'loc': 'upper left'})
    
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    cx.add_basemap(ax, crs=df.crs, source=cx.providers.CartoDB.Positron)

In [ ]:
foo(df, 2263)

In [ ]:
foo(df, 4326)

In [ ]:
import geodatasets as gds
import geopandas as gpd
import geoviews as gv
import cartopy.crs as ccrs

gv.extension('bokeh');

In [ ]:
def bar(df, epsg):
    crs = ccrs.epsg(epsg)
    df = df.to_crs(crs)
    
    tiles = gv.tile_sources.CartoLight
    polys = gv.Polygons(df, vdims=['BoroName'], crs=df.crs)

    out = tiles * polys.opts(color='BoroName', cmap='Pastel1', tools=['hover'], width=1200, height=800, alpha=0.5, active_tools=['wheel_zoom'], xaxis=None, yaxis=None)

    return out

In [ ]:
bar(df, 3857)

# converting to cells

In [ ]:
df = geopandas.read_file(geodatasets.get_path('nybb'))
df = df[['BoroName', 'geometry']]
df = df.to_crs(epsg=4326)
df

In [ ]:
res = 9
df.geometry.apply(lambda x: h3.geo_to_cells(x, res=res)).apply(h3.cells_to_h3shape)

In [ ]:
res = 9
df.geometry.apply(lambda x: h3.geo_to_cells(x, res=res)).apply(h3.cells_to_h3shape)[1]

In [ ]:
res = 9

df2 = df.copy()
df2.geometry = df.geometry.apply(lambda x: h3.geo_to_cells(x, res=res)).apply(h3.compact_cells).apply(lambda x: h3.H3MultiPoly(*[h3.cells_to_h3shape({y})[0] for y in x]))
df2.BoroName = df2.BoroName + ' H3'

In [ ]:
df2

In [ ]:
df3 = pd.concat([df, df2])

In [ ]:
bar(df2, 3857)

In [ ]:
# TODO: this needs to show H3MultiPoly
df['geometry_h3_9'] = df.geometry.apply(lambda x: h3.geo_to_cells(x, 9)).apply(lambda x: h3.cells_to_h3shape(x))
df

In [ ]:
df.geometry = df.geometry_h3_9

In [ ]:
df

In [ ]:
crs = ccrs.epsg(3857)
df = df.to_crs(crs)

tiles = gv.tile_sources.CartoLight
polys = gv.Polygons(df, vdims=['BoroName'], crs=df.crs)

out = tiles * polys.opts(color='BoroName', cmap='Pastel1', tools=['hover'], width=1200, height=800, alpha=0.5, active_tools=['wheel_zoom'], xaxis=None, yaxis=None)

return out

In [ ]:
bar(df, 3857)

old version:

`H3MultiPoly(<H3Poly[368]>, <H3Poly[20]>, <H3Poly[6]>)`

new version:

`<H3MultiPoly([368], [20], [6])>`

In [ ]:
import pandas as pd

pd.DataFrame(out)

In [ ]:
for a in out:
    print(a)

In [ ]:
"<H3Poly: [368]>"

In [ ]:
out = df84.geometry.apply(lambda x: h3.geo_to_cells(x, 9)).apply(lambda x: h3.cells_to_h3shape(x))
out

for a in out:
    out = [p.loopcode for p in a]
    out = ', '.join(out)
    out = f'<H3MultiPoly: {out}>'
    print(out)

In [ ]:
out = df84.geometry.apply(lambda x: h3.geo_to_cells(x, 9)).apply(lambda x: h3.cells_to_h3shape(x))
out

for a in out:
    out = [p.loopcode for p in a]
    out = ', '.join(out)
    out = f'<H3MultiPoly: {out}>'
    print(out)

In [ ]:
def shortcode(poly):
    outer = len(poly.outer)
    holes = tuple(map(len, poly.holes))

    outer = str(outer)

    if holes:
        out = outer + '/' + str(holes)
    else:
        out = outer

    return out

In [ ]:
def my_repr(h3poly):
    polys = [
        str(len(p.outer)) + '/'+ str(tuple(map(len, p.holes)))
        for p in h3poly
    ]
    return 'H3MultiPoly:' + str(polys)

for a in out:
    print(my_repr(a))

In [ ]:
def my_repr(h3poly):
    polys = [
        shortcode(p)
        for p in h3poly
    ]
    return 'H3MultiPoly:' + str(polys)

for a in out:
    print(my_repr(a))

In [ ]:
def my_repr(h3poly):
    polys = [
        shortcode(p)
        for p in h3poly
    ]
    return 'H3MultiPoly: ' + ', '.join(polys)

for a in out:
    print(my_repr(a))

In [ ]:
def my_repr(h3poly):
    polys = [
        str(len(p.outer)) + '/'+ str(tuple(map(len, p.holes)))
        for p in h3poly
    ]
    return 'H3MultiPoly: ' + ', '.join(polys)

for a in out:
    print(my_repr(a))

In [ ]:
def my_repr(h3poly):
    polys = [
        str(len(p.outer)) + '-'+ str(tuple(map(len, p.holes)))
        for p in h3poly
    ]
    return 'H3MultiPoly: ' + ', '.join(polys)

for a in out:
    print(my_repr(a))

In [ ]:
def my_repr(h3poly):
    polys = [
        '['+shortcode(p)+']'
        for p in h3poly
    ]
    return 'H3MultiPoly: ' + ', '.join(polys)

for a in out:
    print(my_repr(a))

In [ ]:
def my_repr(h3poly):
    polys = [
        shortcode(p)
        for p in h3poly
    ]
    return 'H3MultiPoly: ' + ', '.join(polys)

for a in out:
    print(my_repr(a))

In [ ]:
# todo: make this the polygon shortcode representation! people will learn fast. make it well-documented. it'll be fine
# example question

"""
Q: what do codes of the form `[n/(x,y,z)]` mean?
A: a polygon with an outer ring of n unique vertices, and 3 holes with x,y,and z unique vertices, respectively.
"""

